# Lab 1 — Local Environment Setup (LoRA + Unsloth)

This notebook prepares a local Python environment for fine‑tuning Small Language Models (SLMs) with **LoRA + Unsloth** on an NVIDIA GPU.
It assumes Linux or WSL2 with a recent NVIDIA driver and Conda installed.

## 0) Prerequisites (install manually before running)
- Miniconda or Anaconda
- NVIDIA GPU driver (verify with `nvidia-smi`)
- Optional: CUDA Toolkit (not required if using PyTorch CUDA wheels)
- Python 3.10 environment
- Jupyter Notebook or VS Code with Jupyter extension

Recommended hardware: 8GB+ VRAM GPU, 16GB+ RAM, ~20GB free disk.

## 1) Verify GPU and drivers

In [ ]:
!nvidia-smi || echo "No GPU detected. Ensure NVIDIA drivers are installed."

## 2) Create and activate Conda environment (run in terminal)
Run these in your terminal and reopen this notebook from that environment:

```
conda create -n unsloth_env python=3.10 -y
conda activate unsloth_env
jupyter notebook
```

## 3) Install a CUDA‑matched PyTorch build
Pick ONE cell below that matches your CUDA version shown by `nvidia-smi`.

In [ ]:
# CUDA 12.1 wheels:
# Uncomment to run:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# CUDA 11.8 wheels:
# Uncomment to run:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# CPU‑only:
# Uncomment to run:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

### Verify PyTorch & CUDA

In [ ]:
import torch, sys
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU. GPU is strongly recommended for training.")

## 4) Install LLM libraries (without changing Torch)
We use `--no-deps` so pip does not upgrade Torch behind the scenes.

In [ ]:
!pip install -U unsloth unsloth_zoo accelerate transformers peft datasets bitsandbytes sentencepiece trl --no-deps

## 5) Create a project scaffold (optional)

In [ ]:
import os
base = os.path.expanduser("~/slm-dragon-labs")
for d in ["data", "scripts", "models", "notebooks"]:
    os.makedirs(os.path.join(base, d), exist_ok=True)
print("Project folders created under:", base)

## 6) Quick model load test (Unsloth)
If you hit permission issues on other repos, this Mistral variant is public.

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = torch.float16
model_name = "unsloth/mistral-7b-v0.2"  # public

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,  # requires bitsandbytes + GPU
)

FastLanguageModel.for_inference(model)
print("Loaded:", model_name)

## 7) Inference smoke test

In [ ]:
prompt = "What is the capital of France?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
outputs = model.generate(**inputs, max_new_tokens=32)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

---

## Troubleshooting
- Reinstall matched Torch build (Step 3) if Torch import fails, then rerun Step 4.
- Use WSL2 or Linux for bitsandbytes; native Windows isn't supported.
- Ensure Step 4 includes both unsloth_zoo and trl.
- If GPU not detected, (re)install NVIDIA drivers and relaunch the Conda env.